In [ ]:
import pandas as pd
import geopandas as gpd
import subprocess
import numpy as np
import shapely.geometry as shpg
import os
import sys
import json
import logging

In [ ]:
sys.path.append(os.getcwd() + '/../..')
from utils import mkdir, open_zip_shapefile, open_tar_shapefile, haversine

In [ ]:
log = logging.getLogger('papermill')
logging.basicConfig(level='INFO', format="%(message)s")

## Files and storage paths

In [ ]:
# Region of interest
reg = 2

# go down from rgi7_scripts/workflow
data_dir = '../../../../rgi7_data/'

# Input dirctory
input_dir_tar = os.path.join(data_dir, 'l4_rgi7b0_tar')
input_dir = os.path.join(data_dir, 'l4_rgi7b0')

### Load the input data

In [ ]:
reg_file = os.path.join(data_dir, '00_rgi70_regions', '00_rgi70_O1Regions')
reg_file = gpd.read_file(reg_file)
fname = reg_file.loc[reg_file['o1region'] == f'{reg:02d}'].iloc[0]['long_code']

In [ ]:
# Read beta files
shp = open_tar_shapefile(input_dir_tar + f'/RGI2000-v7.0-G-{fname}.tar.gz')
odf_subm = pd.read_csv(input_dir + f'/RGI2000-v7.0-G-{fname}/RGI2000-v7.0-G-{fname}_subm_info.csv', index_col=0)

## Printouts for docs 

In [ ]:
for i, line in odf_subm.iterrows():
    print(f"Submission {i}")
    print(f": **Submitter**: {line['submitters']}."
          f"<br/>**Number of outlines**: {line['n_outlines']}. **Area**: {line['area_km2']:.1f}km². **Release date**: {line['release_dt'][:10]}."
          f"<br/>**Analysts**: {line['analysts']}.")
    print(f"")

## Maps 

In [ ]:
import salem
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import cartopy.crs as ccrs
from matplotlib.image import imread
import seaborn as sns

In [ ]:
rdf = salem.read_shapefile(os.path.join(data_dir, '00_rgi70_regions', '00_rgi70_O1Regions', '00_rgi70_O1Regions.shp'))
srdf = salem.read_shapefile(os.path.join(data_dir, '00_rgi70_regions', '00_rgi70_O2Regions', '00_rgi70_O2Regions.shp'))

rdf = rdf.loc[rdf['o1region'] == f'{reg:02d}']
srdf = srdf.loc[srdf['o1region'] == f'{reg:02d}']

In [ ]:
extent = [rdf.min_x.iloc[0], rdf.max_x.iloc[0], rdf.min_y.iloc[0], rdf.max_y.iloc[0]]
extent

In [ ]:
extent = [-136, -102.0, 32.0, 75.0]

In [ ]:
map_crs = ccrs.LambertConformal(
    central_longitude=(extent[0] + extent[1]) / 2,
    central_latitude=(extent[2] + extent[3]) / 2,
    standard_parallels=(40, 60),
)

In [ ]:
shp_repr = shp.to_crs(map_crs)

In [ ]:
for is_rgi6 in [False, True]:
    linewidth = 0.2
    title = 'RGI 02 - Western Canada and USA' 

    fig = plt.figure(figsize=(6, 10))
    ax = fig.add_subplot(1, 1, 1, projection=map_crs)
    ax.set_extent(extent, crs=ccrs.Geodetic())

    gl = ax.gridlines(draw_labels=True, x_inline=False, y_inline=False, dms=True, 
                      xlocs=np.arange(37) * 10 - 180);
    gl.top_labels = False
    gl.right_labels = False

    # fname = '/home/users/fmaussion/www_fmaussion/misc/rgi7_data/l0_support_data/images/natural-earth-1_large4096px.png'
    fname = '/home/users/fmaussion/www_fmaussion/misc/rgi7_data/l0_support_data/images/natural-earth-1_large8192px_white.png'
    ax.imshow(imread(fname), origin='upper', transform=ccrs.PlateCarree(), 
              extent=[-180, 180, -90, 90])

    rdf.to_crs(map_crs).plot(ax=ax, edgecolor='k', facecolor='none', zorder=99, linewidth=1.5)
    srdf.to_crs(map_crs).plot(ax=ax, edgecolor='k', facecolor='none', zorder=98, linewidth=0.5)

    handles = []
    if is_rgi6:
        for itis in [True, False]:
            ec = '#001fff' if itis else '#ff0000'
            fc = '#465dff' if itis else '#ff4646'
            label = 'Same as RGI6' if itis else 'New in RGI7'
            s_loc = shp_repr.loc[shp_repr['is_rgi6'] == itis]
            if len(s_loc) > 0:
                s_loc.plot(ax=ax, facecolor=fc, edgecolor=ec, linewidth=linewidth, zorder=97)
                area = s_loc['area_km2'].sum()
            else:
                area = 0
            print(f'N={len(s_loc)}, A={area:.1f} km²')
            handles.append(mpatches.Patch(facecolor=ec, label=f'{label}'))
    else:
        for i, subid in enumerate(sorted(shp.subm_id.unique().astype(int))):
            fc = f'C{i}'
            ec = fc
            s_loc = shp_repr.loc[shp.subm_id == subid]
            s_loc.plot(ax=ax, facecolor=fc, edgecolor=ec, linewidth=linewidth, zorder=97)
            area = s_loc['area_km2'].sum()
            handles.append(mpatches.Patch(facecolor=fc, label=f'{subid}'))
            print(f'N={len(s_loc)}, A={area:.1f} km²')

    # Alternative solution for "Polygon handles not passing to legend"
    ax.legend(handles=handles, loc='upper left');
    ax.set_title(title);

    plot_dir = data_dir + f'/l4_rgi7b0_plots/RGI{reg:02d}'
    mkdir(plot_dir)
    plotname = 'isrgi6_map' if is_rgi6 else 'inventory_map'
    plt.savefig(plot_dir + f'/{plotname}.jpeg', bbox_inches='tight', dpi=300)

In [ ]:
figsize=(8, 7)

f, (ax1, ax2) = plt.subplots(2, 1, figsize=figsize)

date_i = shp['src_date'].str[:4].astype(int).to_frame('src_date')
date_i['area'] = shp['area_km2']
date = date_i.groupby('src_date')['area'].sum().to_frame()
date['area'] = date['area'] / date['area'].sum()
date['count'] =  date_i.groupby('src_date')['area'].count()
date = date.reset_index()

sns.barplot(ax=ax1, x='src_date', y='area', data=date, color='lightsteelblue');
ax1.set_ylabel('Relative area'); ax1.set_xlabel(''); 
ax1.set_title(title);
for tick in ax1.get_xticklabels():
    tick.set_rotation(45)

sns.barplot(ax=ax2, x='src_date', y='count', data=date, color='thistle');
ax2.set_ylabel('Number of outlines'); ax2.set_xlabel('Source date'); 
plt.xticks(rotation=45, ha='center')

plot_dir = data_dir + f'/l4_rgi7b0_plots/RGI{reg:02d}'
mkdir(plot_dir)
plotname = 'date_hist'
plt.tight_layout()
plt.savefig(plot_dir + f'/{plotname}.png', bbox_inches='tight', dpi=300)